### Try-it 9.2: Predicting Wages

This activity is meant to summarize your work with regularized regression models.  You will use your earlier work with data preparation and pipelines together with what you've learned with grid searches to determine an optimal model.  In addition to the prior strategies, this example is an excellent opportunity to utilize the `TransformedTargetRegressor` estimator in scikitlearn.

### The Data

This dataset is loaded from the OpenML resource library.  Originally from census data, the data contains wage and demographic information on 534 individuals. From the dataset documentation [here](https://www.openml.org/d/534)

```
The Current Population Survey (CPS) is used to supplement census information between census years. These data consist of a random sample of 534 persons from the CPS, with information on wages and other characteristics of the workers, including sex, number of years of education, years of work experience, occupational status, region of residence, and union membership. 
```

In [ ]:
from sklearn.datasets import fetch_openml

In [ ]:
wages = fetch_openml(data_id=534, as_frame=True)

In [ ]:
wages.frame.head()

#### Task

Build regression models to predict `WAGE`.  Incorporate the categorical features and transform the target using a logarithm.  Build `Ridge` models and consider some different amounts of regularization.  

After fitting your model, interpret the model and try to understand what features led to higher wages.  Consider using `permutation_importance` that you encountered in module 8.  Discuss your findings in the class forum.

For an in depth example discussing the perils of interpreting the coefficients, see the example in scikitlearn examples [here](https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html).

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from sklearn.compose import TransformedTargetRegressor, make_column_selector, ColumnTransformer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
df = wages.frame
df.info()
df.columns = df.columns.str.lower()
df.info()

In [ ]:
X = df.drop('wage', axis=1)
y = df['wage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

In [ ]:
preprocessing = ColumnTransformer(
    [('cat', OneHotEncoder(), make_column_selector(dtype_include="category")),
     ('num', 'passthrough', make_column_selector(dtype_exclude="category"))])

In [ ]:
linreg_pipe = Pipeline([
    ('preprocessing', preprocessing),
    ('poly', PolynomialFeatures(include_bias=False)),
    ('seq', SequentialFeatureSelector(estimator=LinearRegression())),
    ('linreg', LinearRegression())
])

param_grid = {
    'seq__n_features_to_select': [3, 4, 5, 6],
    'poly__degree': [2, 3, 4],
}

grid_linreg = GridSearchCV(estimator=linreg_pipe, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1)
grid_linreg.fit(X_train, y_train)

In [ ]:
linreg_train_mse = mean_squared_error(grid_linreg.best_estimator_.predict(X_train), y_train)
linreg_test_mse = mean_squared_error(grid_linreg.best_estimator_.predict(X_test), y_test)
print(f'Train MSE: {linreg_train_mse}, Test MSE: {linreg_test_mse}, Best Params: {grid_linreg.best_params_}')

In [ ]:
ridge_pipe = Pipeline([
    ('preprocessing', preprocessing),
    ('poly', PolynomialFeatures(include_bias=False)),
    ('seq', SequentialFeatureSelector(estimator=LinearRegression())),
    ('ridge', Ridge())
])

param_grid = {
    'seq__n_features_to_select': [3, 4, 5, 6],
    'poly__degree': [2, 3, 4],
    'ridge__alpha': [0.01, 0.1, 1, 10],
}

grid_ridge = GridSearchCV(estimator=ridge_pipe, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)
grid_ridge.fit(X_train, y_train)

In [ ]:
ridge_train_mse = mean_squared_error(grid_ridge.best_estimator_.predict(X_train), y_train)
ridge_test_mse = mean_squared_error(grid_ridge.best_estimator_.predict(X_test), y_test)
print(f'Train MSE: {ridge_train_mse}, Test MSE: {ridge_test_mse}, Best Params: {grid_ridge.best_params_}')

In [ ]:
target_pipe = Pipeline([
    ('preprocessing', preprocessing),
    ('poly', PolynomialFeatures(include_bias=False)),
    ('seq', SequentialFeatureSelector(estimator=LinearRegression())),
    ('transform_linreg', TransformedTargetRegressor(LinearRegression(), func=np.log, inverse_func=np.exp)),
])

param_grid = {
    'seq__n_features_to_select': [3, 4, 5, 6],
    'poly__degree': [2, 3, 4],
}

grid_target = GridSearchCV(estimator=target_pipe, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)
grid_target.fit(X_train, y_train)

In [ ]:
target_train_mse = mean_squared_error(grid_target.best_estimator_.predict(X_train), y_train)
target_test_mse = mean_squared_error(grid_target.best_estimator_.predict(X_test), y_test)
print(f'Train MSE: {target_train_mse}, Test MSE: {target_test_mse}, Best Params: {grid_target.best_params_}')